In [ ]:
import os
import glob
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=20)
pwd_path = os.getcwd()
from matplotlib import gridspec
import xarray as xr
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt

path1_SSDP_data = '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final/'
# Calculate the irrigation water demand for each realisations of MICE and MME
path_irrigation_demand_ssp245_historical_MME = '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/CMIP6_preprocessed_gujarat/MME_crop_water_requirements/MME_avg_monthly_cropwater_requirement_cubicmeters_historical_all_realisations.nc'
path_irrigation_demand_ssp245_RCP245_MME ='/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/CMIP6_preprocessed_gujarat/MME_crop_water_requirements/MME_avg_monthly_cropwater_requirement_cubicmeters_rcp245_all_realisations.nc'
path_irrigation_demand_ssp245_historical_MICE = '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/CMIP6_preprocessed_gujarat/MICE_crop_water_requirements/MICE_avg_monthly_cropwater_requirement_cubicmeters_historical_all_realisations.nc'
path_irrigation_demand_ssp245_RCP245_MICE ='/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/CMIP6_preprocessed_gujarat/MICE_crop_water_requirements/MICE_avg_monthly_cropwater_requirement_cubicmeters_rcp245_all_realisations.nc'

streamflow_MME_ssp245 = xr.open_dataset(path1_SSDP_data + 'streamflow_full_montlhy_term_all_initial_condition_narmada_MME_ssp245.nc')
# streamflow_MICE_ssp245 = xr.open_dataset(path1_SSDP_data + 'streamflow_full_montlhy_term_all_initial_condition_narmada_MICE_ssp245.nc')
# streamflow_MME_ssp585 = xr.open_dataset(path1_SSDP_data + 'streamflow_full_term_all_initial_condition_narmada_MME_ssp585.nc')
# streamflow_MICE_ssp585 = xr.open_dataset(path1_SSDP_data +'streamflow_full_term_all_initial_condition_narmada_MICE_ssp585.nc')
domestic_demand= xr.open_dataset(path1_SSDP_data +'drinking_water_requirements_1901_2101_data.nc')
nc_fpaths = [path_irrigation_demand_ssp245_historical_MME,path_irrigation_demand_ssp245_RCP245_MME]
irrigation_water_demand_MME = xr.open_mfdataset(nc_fpaths, coords='minimal',  engine='netcdf4')
nc_fpaths = [path_irrigation_demand_ssp245_historical_MICE,path_irrigation_demand_ssp245_RCP245_MICE]
irrigation_water_demand_MICE = xr.open_mfdataset(nc_fpaths, coords='minimal',  engine='netcdf4')
benefit_calculated = xr.open_dataset(path1_SSDP_data +'benefit_SSDP_different_combinations_nonnegetive_release_irrigation_case1.nc')
bins_irrigation=[0, 150, 295, 336,380,410, 440, 480,630,710,750, 780, 840,965, 1050,  1090,1120, 1180,1270,1450, 1700, 2000,2150,2440,2800, 3350 ]
#bins = (np.array(bins)/1000000) # in MCM
irrigation_demand_classes1 = []
irrigation_demand_representative_values1 = []
for class1 in range(len(bins_irrigation) - 1):
    class2 = [bins_irrigation[class1],bins_irrigation[class1+1]]
    representative_values = (bins_irrigation[class1]+bins_irrigation[class1+1])/2
    #print(representative_values)
    irrigation_demand_representative_values1.append(representative_values)
    irrigation_demand_classes1.append(class2)
print(len(irrigation_demand_representative_values1))

bins_inflow=[0,3,6,20, 50, 100,150, 200,300,400,600,800,1000,1200,1500, 2000,2700, 3500,4500, 5000,6000, 7000,8000,9400,10500,12000,15000,20000, 30000,40000,48300]
inflow_classes1 = []
inflow_representative_values1 = []
for class1 in range(len(bins_inflow) - 1):
    class2 = [bins_inflow[class1],bins_inflow[class1+1]]
    representative_values = (bins_inflow[class1]+bins_inflow[class1+1])/2
    #print(representative_values)
    inflow_representative_values1.append(representative_values)
    inflow_classes1.append(class2)
print(len(inflow_representative_values1))
bins_storage=[0.002,0.3,0.45,0.6,0.75,0.9,1.05,1.14
                                        ,1.2,1.3,1.37, 1.42, 1.48, 1.535, 1.57,2,2.5, 3.01823,3.5, 3.991,4.4,4.8,5, 5.4, 5.65,5.76]
print(len(bins_storage))
bins_storage= np.array(bins_storage)* 1000
storage_classes = []
storage_representative_values = []
for class1 in range(len(bins_storage) - 1):
    class2 = [bins_storage[class1],bins_storage[class1+1]]
    representative_values = (bins_storage[class1]+bins_storage[class1+1])/2
    #print(representative_values)
    storage_representative_values.append(representative_values)
    storage_classes.append(class2)

inflow_full_monthly = streamflow_MME_ssp245
irrigation_water_demand = irrigation_water_demand_MME

domestic_demand = xr.open_dataset(path1_SSDP_data +'drinking_water_requirements_1901_2101_data.nc')
def class_representative_values(inflow1, inflow_classes1,inflow_representative_values1):
    for i in range(len(inflow_classes1)):
        #print(inflow1)
        if inflow_classes1[i][0] <= inflow1 and inflow_classes1[i][1] > inflow1:
            inflow_representative_values2 = inflow_representative_values1[i]
    return(inflow_representative_values2)
        
def SDP_transition_probability(filtered_df, month_1, month_2, transition_probability_name):
    transition_probability = np.zeros((len(inflow_representative_values1), len(inflow_representative_values1)))
    jan_feb_pairs_1 = []
    for i, year in enumerate(total_years):
        data1 = filtered_df['streamflow'].loc[(filtered_df['streamflow'].index.month == month_1) & (filtered_df['streamflow'].index.year == year)].values[0]
        if month_1 == 12 and i == len(total_years)-1:
            data2 = filtered_df['streamflow'].loc[(filtered_df['streamflow'].index.month == month_2) & (filtered_df['streamflow'].index.year == total_years[0])].values[0]
        elif month_1 == 12 and i != len(total_years)-1:
            data2 = filtered_df['streamflow'].loc[(filtered_df['streamflow'].index.month == month_2) & (filtered_df['streamflow'].index.year == year+1)].values[0]
        else:
            data2 = filtered_df['streamflow'].loc[(filtered_df['streamflow'].index.month == month_2) & (filtered_df['streamflow'].index.year == year)].values[0]
            
        data1_class = class_representative_values(data1, inflow_classes1,inflow_representative_values1)
        data2_class = class_representative_values(data2, inflow_classes1,inflow_representative_values1)
        jan_feb_pairs = (data1_class, data2_class)
        jan_feb_pairs_1.append(jan_feb_pairs)

    for pair in jan_feb_pairs_1:
        data1_class, data2_class = pair
        index1 = inflow_representative_values1.index(data1_class)
        index2 = inflow_representative_values1.index(data2_class)
        transition_probability[index1, index2] += 1

    # Normalize the transition probability matrix
    transition_probability /= np.sum(transition_probability, axis=1, keepdims=True)
    transition_probability = np.nan_to_num(transition_probability)
    df = pd.DataFrame(transition_probability)
    df.to_excel(transition_probability_name, index=False)



In [ ]:
for ini_condition_number in tqdm(itertools.islice(range(len(inflow_full_monthly['initial_condition'].values)),0,1,1)):
#for ini_condition_number in range(len(inflow_full_monthly['initial_condition'])):
    inflow_full_monthly_1 = inflow_full_monthly.sel(initial_condition = inflow_full_monthly['initial_condition'][ini_condition_number])
    initial_condition = inflow_full_monthly['initial_condition'][ini_condition_number]
    #print(inflow_full_monthly_1)
    initial_condition1 = initial_condition.values
    series = inflow_full_monthly_1['streamflow'].to_series()
    # Create a DatetimeIndex from 'year' and 'month' coordinates
    date_index = pd.to_datetime(dict(year=series.index.get_level_values('year'),
                                     month=series.index.get_level_values('month'),
                                     day=1))
    # Set the date index for the series
    series.index = date_index
    # Convert the series to a pandas DataFrame
    df = pd.DataFrame(series, columns=['streamflow'])
    # Filter the data for the desired date range
    start_date = '1981-01-01'
    end_date = '2014-12-31'
    filtered_df = df.loc[start_date:end_date]
    total_years = (np.unique(filtered_df['streamflow'].index.year))
    # calculation of transition probability from one month to another month for each realisation
#     SDP_transition_probability(filtered_df, 1, 2, pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_1_2_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 2, 3, pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_2_3_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 3, 4, pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_3_4_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 4, 5,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_4_5_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 5, 6,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_5_6_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 6, 7,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_6_7_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 7, 8,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_7_8_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 8, 9,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_8_9_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 9, 10,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_9_10_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 10, 11,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_10_11_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 11, 12,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_11_12_'+str(initial_condition1)+'.xlsx')
#     SDP_transition_probability(filtered_df, 12, 1,pwd_path+'/SDP_MME_245_benefit_optimised_data/transition_probability_12_1_'+str(initial_condition1)+'.xlsx')
    # calculation for 12th month of 2014
    abc11 = xr.open_dataset(pwd_path+'/combination_s_t_inflow_t1_and_s_T_1_allcases.nc')
    abc11 = abc11.to_dataframe().reset_index()
    domestic_demand_data = (domestic_demand['Drinking_water_demand'].sel(year = 2014).values)/1000000
    if initial_condition1 == 'EC-EARTH3':
        initial_condition2 = 'EC-Earth3'
        irrigation_water_demand_data = (irrigation_water_demand['GIR_crop_volume_cubicmeter'].sel(realisation = initial_condition2, year = 2014,month =12).values)/1000000
        irrigation_water_demand_data = class_representative_values(irrigation_water_demand_data, irrigation_demand_classes1,irrigation_demand_representative_values1)
    else:
        irrigation_water_demand_data = (irrigation_water_demand['GIR_crop_volume_cubicmeter'].sel(realisation = initial_condition1, year = 2014,month =12).values)/1000000
        irrigation_water_demand_data = class_representative_values(irrigation_water_demand_data, irrigation_demand_classes1,irrigation_demand_representative_values1)
                    #print(initial_condition1)
    if initial_condition1 == 'EC-EARTH3':
        initial_condition2 = 'EC-EARTH3'
        inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition2, year = 2014,month =12).values
        inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
    else:
        inflow_t1 = inflow_full_monthly['streamflow'].sel(initial_condition = initial_condition1, year = 2014,month =12).values
        inflow_t1 = class_representative_values(inflow_t1, inflow_classes1,inflow_representative_values1)
    benefit_calculated = xr.open_dataset(path1_SSDP_data +'benefit_SSDP_different_combinations_nonnegetive_release_irrigation_case1.nc')
#########################################################################################################################    
    # calculation of benefit for last time period
    # T = 2014
#     df_benefit = pd.DataFrame({'S_t':[], 'S_t_1':[], 'inflow_t':[],'benefit':[],'Release_t':[],'drinking_water_release':[],'irrigation_water_release':[],'Industrial_water_release':[],'power_production':[]})
#     for combination_1_number in tqdm(itertools.islice(range(len(abc11['S_t'].values)),0,None,1)):
#         benefit = - (benefit_calculated['benefit_calculated'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand_data,month = 11).values)
#         benefit_scaled = (benefit/(10 ** 4))
#         Release_t_scaled = (benefit_calculated['Release_t'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand_data,month = 11).values)
#         drinking_water_release_scaled = (benefit_calculated['drinking_water_release'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand_data,month = 11).values)
#         irrigation_water_release_scaled = (benefit_calculated['irrigation_water_release'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand_data,month = 11).values)
#         Industrial_water_release_scaled = (benefit_calculated['Industrial_water_release'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand_data,month = 11).values)
#         power_production_scaled = (benefit_calculated['power_production'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand_data,month = 11).values)
#     # Append the values to the DataFrame
#         df_benefit = df_benefit.append({
#             'S_t': abc11['S_t'][combination_1_number],
#             'S_t_1': abc11['S_t_1'][combination_1_number],
#             'inflow_t': abc11['inflow_t1'][combination_1_number],
#             'benefit': benefit_scaled,
#             'Release_t': Release_t_scaled,
#             'drinking_water_release': drinking_water_release_scaled,
#             'irrigation_water_release': irrigation_water_release_scaled,
#             'Industrial_water_release': Industrial_water_release_scaled,
#             'power_production': power_production_scaled
#         }, ignore_index=True)

#     # Print the DataFrame
#     df_benefit.set_index(['S_t','S_t_1', 'inflow_t'], inplace=True)
#     print(df_benefit)
#     benefit_values_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'benefit_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+'_2014_12.nc'
#     df_benefit.to_xarray().to_netcdf(benefit_values_name)
    #open calculated benefit data to calculate the optimised data for last year
#    del abc11
    benefit_values_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'benefit_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+'_2014_12.nc'
    df3 = xr.open_dataset(benefit_values_name)
    #check
#     print(df3['benefit'].max()) # value should be -0.21788245340839835
#     print(df3['benefit'].min()) # value should be -101000632.70706701
########################################################################################################################
# calculation for optimised values
#     abc12 = xr.open_dataset(pwd_path+'/combination_s_t_and_inflow_t1_allcases.nc')
#     abc12 = abc12.to_dataframe().reset_index()
#     df_optimised = pd.DataFrame({'S_t':[], 'inflow_t':[],'optimised_function_value':[],'optimised_storage_t':[],'optimised_drinking_release':[],'optimised_irrigation_release':[],'optimised_industrial_release':[],'optimised_power_production':[]})
#     for combination_2_number in tqdm(itertools.islice(range(len(abc12['S_t'].values)),0,None,1)):
#         data_index = df3['benefit'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number]).values
#         max_value = np.max(data_index)
#         max_indices = np.argwhere(data_index == max_value).flatten().tolist()
#         if len(max_indices) >1:
#             max_index1 = np.min(max_indices)
#         else:
#             max_index1  = max_indices[0]
#         #print(max_index1)
#         opti_function_value = df3['benefit'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
#         #print(opti_function_value)
#         opti_storage_t = df3['S_t_1'].values[max_index1]
#         opti_drinking_release = df3['drinking_water_release'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
#         opti_irrigation_release = df3['irrigation_water_release'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
#         opti_industrial_release = df3['Industrial_water_release'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
#         opti_power_production = df3['power_production'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
#         # Append the values to the DataFrame
#         df_optimised = df_optimised.append({
#             'S_t': abc12['S_t'][combination_2_number],
#             'inflow_t': abc12['inflow_t1'][combination_2_number],
#             'optimised_function_value': opti_function_value,
#             'optimised_storage_t': opti_storage_t,
#             'optimised_drinking_release': opti_drinking_release,
#             'optimised_irrigation_release': opti_irrigation_release,
#             'optimised_industrial_release': opti_industrial_release,
#             'optimised_power_production': opti_power_production
#             }, ignore_index=True)
#     df_optimised.set_index(['S_t', 'inflow_t'], inplace=True)
#     optimised_values_name1 = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'optimised_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+'_2014_12.nc'
#     df_optimised.to_xarray().to_netcdf(optimised_values_name1)

In [ ]:
# calculation for the remaining time period

In [ ]:
modellists = glob.glob(pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'benefit_value_*.nc')
#print(len(modellists))
modellists1 = glob.glob(pwd_path+ '/SDP_MME_245_benefit_optimised_data/'+'optimised_value_*.nc')
#print(len(modellists1))
inflow_full_monthly = streamflow_MME_ssp245
end_year = 2014
years1 = list(range(1980,2015))
months = [1,2,3,4,5,6,7,8,9,10,11,12]
initial_condition_list = inflow_full_monthly['initial_condition'].values
for condition_list, initial_condition in tqdm(itertools.islice(enumerate(initial_condition_list),0,1,1)):
    for year_no in tqdm(reversed(years1),0,None,1):
        for month_no in tqdm(reversed(months),0,None,1):
            file_exists = False
            for number in range(len(modellists)):
                if str(modellists[number].split('/')[-1].split('_')[-3]) == initial_condition and int(modellists[number].split('/')[-1].split('_')[-2]) == year_no and int(modellists[number].split('/')[-1].split('_')[-1].split('.')[0]) == month_no:
                    
                    file_exists = True
                    break
            if not file_exists:
                print('calculation')
                print("year =", year_no)
                print("month =", month_no)
                irrigation_water_demand_ssp245_MME = xr.open_dataset(path_irrigation_demand_ssp245_historical_MME)
                initial_condition1 = initial_condition
                if initial_condition1 == 'EC-EARTH3':
                    initial_condition1 = 'EC-Earth3'
                irrigation_water_demand_data = (irrigation_water_demand_ssp245_MME['GIR_crop_volume_cubicmeter'].sel(realisation = initial_condition1, year = year_no,month =month_no).values)/1000000
                irrigation_water_demand1 = class_representative_values(irrigation_water_demand_data, irrigation_demand_classes1,irrigation_demand_representative_values1)

                abc11 = xr.open_dataset(pwd_path+'/combination_s_t_inflow_t1_and_s_T_1_allcases.nc')
                abc11 = abc11.to_dataframe().reset_index()
                domestic_demand_data = (domestic_demand['Drinking_water_demand'].sel(year = year_no).values)/1000000
                
########################################################################################################################
                # benefit calculation for differnt month and year
                df_benefit = pd.DataFrame({'S_t':[], 'S_t_1':[], 'inflow_t':[],'benefit':[],'Release_t':[],'drinking_water_release':[],'irrigation_water_release':[],'Industrial_water_release':[],'power_production':[]})
                for combination_1_number in tqdm(itertools.islice(range(len(abc11['S_t'].values)),0,None,1)):
                    benefit = - (benefit_calculated['benefit_calculated'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand1,month = 11).values)
                    benefit_scaled = (benefit/(10 ** 4))
                    Release_t_scaled = (benefit_calculated['Release_t'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand1,month = 11).values)
                    drinking_water_release_scaled = (benefit_calculated['drinking_water_release'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand1,month = 11).values)
                    irrigation_water_release_scaled = (benefit_calculated['irrigation_water_release'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand1,month = 11).values)
                    Industrial_water_release_scaled = (benefit_calculated['Industrial_water_release'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand1,month = 11).values)
                    power_production_scaled = (benefit_calculated['power_production'].sel(S_t= abc11['S_t'][combination_1_number],S_t_1=abc11['S_t_1'][combination_1_number],inflow_t = abc11['inflow_t1'][combination_1_number],drinking_water_demand1 = domestic_demand_data,irrigation_water_demand1 = irrigation_water_demand1,month = 11).values)
                # Append the values to the DataFrame
                    df_benefit = df_benefit.append({
                        'S_t': abc11['S_t'][combination_1_number],
                        'S_t_1': abc11['S_t_1'][combination_1_number],
                        'inflow_t': abc11['inflow_t1'][combination_1_number],
                        'benefit': benefit_scaled,
                        'Release_t': Release_t_scaled,
                        'drinking_water_release': drinking_water_release_scaled,
                        'irrigation_water_release': irrigation_water_release_scaled,
                        'Industrial_water_release': Industrial_water_release_scaled,
                        'power_production': power_production_scaled
                    }, ignore_index=True)

                # Print the DataFrame
                df_benefit.set_index(['S_t','S_t_1', 'inflow_t'], inplace=True)
                print(df_benefit)
                benefit_values_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'benefit_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+ '_'+str(year_no) + '_'+ str(month_no)+'.nc'
                df_benefit.to_xarray().to_netcdf(benefit_values_name)
                del abc11
                del df_benefit
########################################################################################################################
                # calculation for optimised data for all t-1 years
                benefit_values_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'benefit_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+ '_'+str(year_no) + '_'+ str(month_no)+'.nc'
                benefit_values = xr.open_dataset(benefit_values_name)
                if month_no == 12:
                    optimised_values_name_nexttime = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'optimised_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+ '_'+str(year_no+1) + '_1.nc'
                else: 
                    optimised_values_name_nexttime = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'optimised_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+ '_'+str(year_no) + '_'+ str(month_no+1)+'.nc'
                optimised_values_nexttime = xr.open_dataset(optimised_values_name_nexttime)
                # calculation for optimised values
                abc12 = xr.open_dataset(pwd_path+'/combination_s_t_and_inflow_t1_allcases.nc')
                abc12 = abc12.to_dataframe().reset_index()
                if month_no == 12:
                    transition_probability_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/transition_probability_12_1_' + str(initial_condition1) + '.xlsx'
                else:
                    transition_probability_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/transition_probability_' + str(month_no) + '_' + str(month_no+1) + '_' + str(initial_condition1) + '.xlsx'
                transition_probability1 = pd.read_excel(transition_probability_name) 
                df_optimised = pd.DataFrame({'S_t':[], 'inflow_t':[],'optimised_function_value':[],'optimised_storage_t':[],'optimised_drinking_release':[],'optimised_irrigation_release':[],'optimised_industrial_release':[],'optimised_power_production':[]})
                for combination_2_number in tqdm(itertools.islice(range(len(abc12['S_t'].values)),0,None,1)):
                    total_all = []
                    for l in tqdm(itertools.islice(range(len(benefit_values['S_t_1'].values)),0,None,1)):
                        bkilt1 = benefit_values['benefit'].sel(S_t = abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number], S_t_1=benefit_values['S_t_1'][l]).values
                        inflow_t1_values = benefit_values['inflow_t'].values
                        inflow_indices = np.where(inflow_t1_values == abc12['inflow_t1'][combination_2_number])
                        f_values = optimised_values_nexttime['optimised_function_value'].sel(S_t=optimised_values_nexttime['S_t'][l], inflow_t=optimised_values_nexttime['inflow_t']).values
                        p_values = transition_probability1.iloc[inflow_indices[0][0]].values
                        second_term_sum = np.sum(p_values * f_values)
                        total_0 = bkilt1 + second_term_sum
                        total_all.append(total_0)
                    #print(total_all)
                    max_value = np.max(total_all)
                    max_indices = np.argwhere(total_all == max_value).flatten().tolist()
                    #print(max_indices)
                    if len(max_indices) >1:
                        max_index1 = np.min(max_indices)
                    else:
                        max_index1  = max_indices[0]
                    #print(max_index1)
                    opti_function_value = benefit_values['benefit'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
                    opti_storage_t = benefit_values['S_t_1'].values[max_index1]
                    opti_drinking_release = benefit_values['drinking_water_release'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
                    opti_irrigation_release = benefit_values['irrigation_water_release'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
                    opti_industrial_release = benefit_values['Industrial_water_release'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
                    opti_power_production = benefit_values['power_production'].sel(S_t= abc12['S_t'][combination_2_number],inflow_t = abc12['inflow_t1'][combination_2_number],S_t_1=df3['S_t_1'][max_index1]).values
                        # Append the values to the DataFrame
                    df_optimised = df_optimised.append({
                        'S_t': abc12['S_t'][combination_2_number],
                        'inflow_t': abc12['inflow_t1'][combination_2_number],
                        'optimised_function_value': opti_function_value,
                        'optimised_storage_t': opti_storage_t,
                        'optimised_drinking_release': opti_drinking_release,
                        'optimised_irrigation_release': opti_irrigation_release,
                        'optimised_industrial_release': opti_industrial_release,
                        'optimised_power_production': opti_power_production
                        }, ignore_index=True)
                df_optimised.set_index(['S_t', 'inflow_t'], inplace=True)
                optimised_values_name1 = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'optimised_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition1)+ '_'+str(year_no) + '_'+ str(month_no)+'.nc'
                df_optimised.to_xarray().to_netcdf(optimised_values_name1)

In [ ]:
# check_optimality

In [ ]:
import os
import glob
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True,nb_workers=30)
pwd_path = os.getcwd()
from matplotlib import gridspec
import xarray as xr
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
path1_SSDP_data = '/home/udit/Desktop/Data_MIR_students/PhD/Divya/divya_data/SSDP_MME_MICE_for_different_cases_final/'

streamflow_MME_ssp245 = xr.open_dataset(path1_SSDP_data + 'streamflow_full_montlhy_term_all_initial_condition_narmada_MME_ssp245.nc')
inflow_full_monthly = streamflow_MME_ssp245

In [ ]:
modellists = glob.glob(pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'difference_between_*.nc')
print(len(modellists))
modellists1 = glob.glob(pwd_path+ '/SDP_MME_245_benefit_optimised_data/'+'difference_of_difference_between_*.nc')
print(len(modellists1))
end_year = 2014
years1 = list(range(1980,2014))
months = [1,2,3,4,5,6,7,8,9,10,11,12]
initial_condition_list = inflow_full_monthly['initial_condition'].values
for condition_list, initial_condition in tqdm(itertools.islice(enumerate(initial_condition_list),0,None,1)):
    if initial_condition == 'EC-EARTH3':
        initial_condition = 'EC-Earth3'
    for year_no in tqdm(reversed(years1),0,1,1):
        file_exists = False
        for number in range(len(modellists)):
            if str(modellists[number].split('/')[-1].split('_')[-3]) == initial_condition and int(modellists[number].split('/')[-1].split('_')[-2]) == year_no:
                file_exists = True
                break
        if not file_exists:
            print('process')

            df_find_difference = pd.DataFrame({'S_t':[], 'inflow':[],'month':[], 'difference':[]})
            for month_no in tqdm(reversed(months),0,1,1):
                optimised_name_currenttime_1 = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'optimised_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition)+ '_'+str(year_no) + '_'+str(month_no)+'.nc'
                optimised_currenttime_1 = xr.open_dataset(optimised_name_currenttime_1)
                optimised_name_nexttime_1 = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'optimised_value_with20storage_nonnegetive_release_MME_SSP245_2014_'+str(initial_condition)+ '_'+str(year_no+1) +'_'+str(month_no)+'.nc'
                optimised_nexttime_1 = xr.open_dataset(optimised_name_nexttime_1)
                for x, storage_no in enumerate(optimised_nexttime_1['S_t']):
                    for y, inflow_no in enumerate(optimised_nexttime_1['inflow_t']):
                        #print(storage_no)
                        value1 = optimised_nexttime_1['optimised_function_value'].sel(S_t = storage_no, inflow_t = inflow_no).values
                        value2 = optimised_currenttime_1['optimised_function_value'].sel(S_t = storage_no, inflow_t = inflow_no).values
                        diff_values = value2 - value1

                    # Append the values to the DataFrame
                        df_find_difference = df_find_difference.append({
                            'S_t': np.float(storage_no.values),
                            'inflow': np.float(inflow_no.values),
                            'month': month_no,
                            'difference': diff_values,
                        }, ignore_index=True)
                        # check print(df_find_difference.dtypes)
            dataset_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'difference_between_'+str(initial_condition) + '_'+str(year_no)+'_'+str(year_no + 1)+'.nc'
            df_find_difference.set_index(['S_t','inflow', 'month'], inplace=True)
            #         # Save the dataset to a NetCDF file
            df_find_difference.to_xarray().to_netcdf(dataset_name)
    
    

In [ ]:
years1 = list(range(1980,2013))
for condition_list, initial_condition in tqdm(itertools.islice(enumerate(initial_condition_list),0,None,1)):
    if initial_condition == 'EC-EARTH3':
        initial_condition = 'EC-Earth3'
    for year_no in tqdm(reversed(years1),0,1,1):
        file_exists = False
        for number in range(len(modellists1)):
            if str(modellists1[number].split('/')[-1].split('_')[-3]) == initial_condition and int(modellists1[number].split('/')[-1].split('_')[-2]) == year_no:
                file_exists = True
                break
        if not file_exists:
            print('process')   
            for year_no in tqdm(reversed(years1),0,1,1):
                dataset_name_difference_1 = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'difference_between_'+str(initial_condition) + '_'+str(year_no)+'_'+str(year_no + 1)+'.nc'
                dataset_difference_1 = xr.open_dataset(dataset_name_difference_1)
                dataset_name_difference_2 = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'difference_between_'+str(initial_condition) + '_'+str(year_no+1)+'_'+str(year_no + 2)+'.nc'
                dataset_difference_2 = xr.open_dataset(dataset_name_difference_2)
                difference_11 = dataset_difference_1['difference'] - dataset_difference_2['difference']
                dataset_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'difference_of_difference_between_'+str(initial_condition) + '_'+str(year_no)+'_'+str(year_no + 1)+'.nc'
                difference_11.to_netcdf(dataset_name)
        

In [ ]:

years1 = list(range(1980,2013))
modellists2 = glob.glob(pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'counts_above_limit_for_*.nc')
for condition_list, initial_condition in tqdm(itertools.islice(enumerate(initial_condition_list),0,None,1)):
    file_exists = False
    for number in range(len(modellists2)):
        if initial_condition == 'EC-EARTH3':
            initial_condition = 'EC-Earth3'
        if str(modellists2[number].split('/')[-1].split('_')[-3]) == initial_condition:
            file_exists = True
            break
    if not file_exists:
        print('process') 
        print(initial_condition)
            
        list12 = []
        len_list12 = []
        list12_1 = []
        df11 = pd.DataFrame({'year':[], 'len_list12_10':[],'len_list12_50':[], 'len_list12_100':[],'len_list12_200':[]})
        for year_no in tqdm(reversed(years1),0,None,1):
            list12_10 = []
            list12_50 = []
            list12_100 = []
            list12_200 = []
            dataset_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'difference_of_difference_between_'+str(initial_condition) + '_'+str(year_no)+'_'+str(year_no + 1)+'.nc'
            dataset_difference_1 = xr.open_dataset(dataset_name)
            for k in tqdm(range(len(dataset_difference_1['S_t'])),0,None,1):
                for i in tqdm(range(len(dataset_difference_1['inflow'])),0,None,1):
                    for month1 in tqdm(range(12),0,None,1):
                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) >= 10:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_10.append(list11)
                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) <= -10:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_10.append(list11)
                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) >= 50:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_50.append(list11)
                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) <= -50:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_50.append(list11)

                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) >= 100:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_100.append(list11)
                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) <= -100:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_100.append(list11)
                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) >= 200:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_200.append(list11)
                        if (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i] ).values) <= -200:
                            list11 = (dataset_difference_1['difference'].sel(month = month1+1, S_t = dataset_difference_1['S_t'][k], inflow = dataset_difference_1['inflow'][i]).values)
                            list12_200.append(list11)
                        else:
                            a =1
                #print(list12)               
                # Append the values to the DataFrame
            df11 = df11.append({'year': year_no, 'len_list12_10':len(list12_10),'len_list12_50':len(list12_50), 'len_list12_100':len(list12_100),'len_list12_200':len(list12_200)}, ignore_index=True)
                                    # check print(df_find_difference.dtypes)
        dataset_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'counts_above_limit_for_'+str(initial_condition) + '_'+str(year_no)+'_'+str(year_no + 1)+'.nc'
        df11.set_index(['year'], inplace=True)
                        #         # Save the dataset to a NetCDF file
        df11.to_xarray().to_netcdf(dataset_name)

          


In [ ]:
years1 = list(range(1980,2013))
modellists2 = glob.glob(pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'counts_above_limit_for_*.nc')
for condition_list, initial_condition in tqdm(itertools.islice(enumerate(initial_condition_list),0,None,1)):
    file_exists = False
    for number in range(len(modellists2)):
        if initial_condition == 'EC-EARTH3':
            initial_condition = 'EC-Earth3'
        if str(modellists2[number].split('/')[-1].split('_')[-3]) == initial_condition:
            file_exists = True
            break
    if not file_exists:
        print('process') 
        print(initial_condition)

In [ ]:
initial_condition_list = ['ACCESS-CM2', 'CanESM5' ,'EC-EARTH3', 'GFDL-ESM4', 'MRI-ESM2-0' ,'NorESM2-LM']
optimal_years_MME =[2006, 1990, 2007, 2005, 1987, 2003]
count = 0

color1 = 'r'
color2 = 'c'
color3 = 'g'
color4 = 'b'

plt.close('all')
plt.rcParams.update({'font.size': 25})

    #projection=ccrs.Orthographic(-90, 85)
fig = plt.figure(figsize=(40,18))
gs = gridspec.GridSpec(2, 3)
for condition_list, initial_condition in tqdm(itertools.islice(enumerate(initial_condition_list),0,None,1)):
    print(initial_condition)
    if initial_condition == 'EC-EARTH3':
        initial_condition = 'EC-Earth3'
    dataset_name = pwd_path + '/SDP_MME_245_benefit_optimised_data/'+'counts_above_limit_for_'+str(initial_condition) + '_1980_1981.nc'
    print(dataset_name)
    dataset_difference_1 = xr.open_dataset(dataset_name)

    ax0 = fig.add_subplot(gs[count])
    plt.plot(dataset_difference_1['year'],dataset_difference_1['len_list12_10'], color=color1, label = 'Number of points above 10')
    plt.plot(dataset_difference_1['year'],dataset_difference_1['len_list12_50'], color=color2, label = 'Number of points above 50')
    plt.plot(dataset_difference_1['year'],dataset_difference_1['len_list12_100'], color=color3, label = 'Number of points above 100')
    plt.plot(dataset_difference_1['year'],dataset_difference_1['len_list12_200'], color=color4, label = 'Number of points above 200')
    ax0.set_title(initial_condition, fontweight ='bold', fontsize=30)
    optimised_year_1 = optimal_years_MME[condition_list]
    plt.scatter(optimised_year_1, dataset_difference_1['len_list12_10'].sel(year = optimised_year_1), color='red', s=400)
    plt.scatter(optimised_year_1, dataset_difference_1['len_list12_50'].sel(year = optimised_year_1), color='red', s=400)
    plt.scatter(optimised_year_1, dataset_difference_1['len_list12_100'].sel(year = optimised_year_1), color='red', s=400)
    plt.scatter(optimised_year_1, dataset_difference_1['len_list12_200'].sel(year = optimised_year_1), color='red', s=400)

#     ax0 = fig.add_subplot(gs[count +6])
#     plt.plot(dataset_difference_1['year'][:9],dataset_difference_1['len_list12_10'][:9], color=color1, label = 'Number of points above 10')
#     plt.plot(dataset_difference_1['year'][:9],dataset_difference_1['len_list12_50'][:9], color=color2, label = 'Number of points above 50')
#     plt.plot(dataset_difference_1['year'][:9],dataset_difference_1['len_list12_100'][:9], color=color3, label = 'Number of points above 100')
#     plt.plot(dataset_difference_1['year'][:9],dataset_difference_1['len_list12_200'][:9], color=color4, label = 'Number of points above 200')
    count = count+1
    print(count)
ax0.legend([ 'Number of points above 10','Number of points above 50', 'Number of points above 100','Number of points above 200'],loc = 9, bbox_to_anchor=(-0.8,- 0.1), ncol=4)
plt.savefig('plot_for_finding_optimal_conditions.png')


In [ ]:
# obtaine Reliability, resiliency and vulnerability of optimal policies

In [ ]:
#define relaiability of the data
def reliability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_data):
    # select the data between start and end date
    selected_data = full_data.sel(date=slice(start_date, end_date))
################################Flood reliability#######################################
    reliability_flood_list = []
    for len_1 in range(len(initial_condition_list)):
        storage_list2 = selected_data['opti_storage'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        storage_list2
        failure = 0
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12]
            for j in range(5,9):
                if storage_list1[j] >= target_monthly_storage[j]:
                    failure = failure+1
                        #print(storage_list1[j])
                        #print(target_monthly_storage[j])
        reliability_flood = 1-(failure/len(storage_list2))
        reliability_flood_list.append(reliability_flood)
 #################################Drinking water reliability#############################   
    reliability_drinking_list =[]
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['drinking_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        drinking_failure = 0
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            target_demand  = (domestic_demand['Drinking_water_demand'].sel(year = start_year1).values)/1000000
            start_year1 = start_year1+1
            for j in range(12):
                if storage_list1[j] < target_demand:
                    drinking_failure = drinking_failure+1
        reliability_drinking = 1-(drinking_failure/len(storage_list2))
        reliability_drinking_list.append(reliability_drinking)
        
        ################################  Irrigation water reliability#############################   
    reliability_irrigation_list =[]
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['irrigation_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        irrigation_failure = 0
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(12):
                #print(irrigation_water_demand_data.sel(year = start_year1,month = month_2+1).values)
                target_irrigation = irrigation_water_demand_data['GIR_crop_volume_cubicmeter'].sel(year = start_year1,month = month_2+1).values.min()/1000000
                target_irrigation = class_representative_values(target_irrigation, irrigation_demand_classes1,irrigation_demand_representative_values1)
                if storage_list1[month_2] < target_irrigation:
                    irrigation_failure = irrigation_failure+1
            start_year1 = start_year1+1
        reliability_irrigation = 1-(irrigation_failure/len(storage_list2))
        reliability_irrigation_list.append(reliability_irrigation)
        
        ################################# Industrial water reliability#############################   
    reliability_industrial_list =[]
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['Industrial_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        industrial_failure = 0
        for i in range(0,len(storage_list2)):
            if storage_list2[i] < Industrial_monthly_Demand:
                industrial_failure = industrial_failure+1
        reliability_industrial = 1-(industrial_failure/len(storage_list2))
        reliability_industrial_list.append(reliability_industrial)
        
        ################################# Hydropower reliability#############################   
    reliability_power_full_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand = 1450
        power_failure = 0
        for i in range(0,len(storage_list2)):
            if storage_list2[i] < power_Demand:
                power_failure = power_failure+1
        reliability_power_full_demand = 1-(power_failure/len(storage_list2))
        reliability_power_full_demand_list.append(reliability_power_full_demand)

                ################################# Hydropower reliability (half) #############################   
    reliability_power_half_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand = 1450/2
        power_failure = 0
        for i in range(0,len(storage_list2)):
            if storage_list2[i] < power_Demand:
                power_failure = power_failure+1
        reliability_power_half_demand = 1-(power_failure/len(storage_list2))
        reliability_power_half_demand_list.append(reliability_power_half_demand)

                ################################# Hydropower reliability (monthly) #############################   
    reliability_power_monthly_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand_monthly = [250, 250,250, 250,250, 1450,1450,1450,1450, 250,250,250 ]
        power_failure = 0
        for i in range(0,len(storage_list2),12):
            storage_list21 = storage_list2[i:i+12]
            print(len(storage_list21))
            for month_2 in range(12):
                print(month_2)
                if storage_list21[month_2] < power_Demand_monthly[month_2]:
                    power_failure = power_failure+1
        reliability_power_monthly_demand = 1-(power_failure/len(storage_list2))
        reliability_power_monthly_demand_list.append(reliability_power_monthly_demand)

    return [reliability_flood_list, reliability_drinking_list, reliability_irrigation_list, reliability_industrial_list, reliability_power_full_demand_list, reliability_power_half_demand_list, reliability_power_monthly_demand_list]


In [ ]:
#define resiliency of the data
def resiliency_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_data):
    # select the data between start and end date
    selected_data = full_data.sel(date=slice(start_date, end_date))
################################Flood resiliency#######################################
    resiliency_flood_list = []
    for len_1 in range(len(initial_condition_list)):
        storage_list2 = selected_data['opti_storage'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        failure = 0
        failure_sucess = 0
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12]
            for j in range(5,9):
                if storage_list1[j] >= target_monthly_storage[j]:
                    failure = failure+1
                if storage_list1[j] >= target_monthly_storage[j] and storage_list1[j+1] < target_monthly_storage[j+1]:
                    failure_sucess = failure_sucess+1
                    #print(failure_sucess)
        resiliency_1 = (failure/len(storage_list2))
        resiliency_2 = (failure_sucess/len(storage_list2))
        if resiliency_1 == 0:
            resiliency_flood = 0
        else:
            resiliency_flood = resiliency_2/resiliency_1
        resiliency_flood_list.append(resiliency_flood)
        
 #################################Drinking water resiliency#############################   
    resiliency_drinking_list =[]
    for len_1 in range(len(initial_condition_list)):
        storage_list2 = selected_data['drinking_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        drinking_failure = 0
        drinking_failure_sucess = 0
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            target_demand  = (domestic_demand['Drinking_water_demand'].sel(year = start_year1).values)/1000000
            #target_demand = class_representative_values(target_demand, domestic_demand_classes1,domestic_demand_representative_values1)

            start_year1 = start_year1+1
            if start_year1 == 2101:
                start_year1 = 2100
            for j in range(11):
                if storage_list1[j] < target_demand:
                    drinking_failure = drinking_failure+1
                if storage_list1[j] < target_demand and storage_list1[j+1] >= target_demand:
                    drinking_failure_sucess = drinking_failure_sucess + 1
            j = 11
            target_demand_1  = (domestic_demand['Drinking_water_demand'].sel(year = start_year1).values)/1000000
            #target_demand_1 = class_representative_values(target_demand, domestic_demand_classes1,domestic_demand_representative_values1)
            #print(j)
            #print(start_year1)
            if storage_list1[j] < target_demand:
                drinking_failure = drinking_failure+1
            if storage_list1[j] < target_demand and storage_list2[i+1] >= target_demand_1:
                drinking_failure_sucess = drinking_failure_sucess + 1

        resiliency_1_drinking = (drinking_failure/len(storage_list2))
        resiliency_2_drinking = (drinking_failure_sucess/len(storage_list2))
        if resiliency_1_drinking <= 0:
            resiliency_drinking = 0
        else:
            resiliency_drinking = resiliency_2_drinking/resiliency_1_drinking
        resiliency_drinking_list.append(resiliency_drinking)
        
        ################################  Irrigation water resiliency#############################   
    resiliency_irrigation_list =[]
    for len_1 in range(len(initial_condition_list)):
        
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['irrigation_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        
        irrigation_failure = 0
        irrigation_failure_sucess = 0
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(11):
                target_irrigation = irrigation_water_demand_data['GIR_crop_volume_cubicmeter'].sel(year = start_year1,month = month_2+1).values.min()/1000000
                target_irrigation = class_representative_values(target_irrigation, irrigation_demand_classes1,irrigation_demand_representative_values1)
                if storage_list1[month_2] < target_irrigation:
                    irrigation_failure = irrigation_failure+1
                if storage_list1[month_2] < target_irrigation and storage_list1[month_2+1] >= target_irrigation:
                    irrigation_failure_sucess = irrigation_failure_sucess + 1
            start_year1 = start_year1+1
            if start_year1 == 2101:
                start_year1 = 2100
            month_2 = 11
            target_irrigation = irrigation_water_demand_data['GIR_crop_volume_cubicmeter'].sel(year = start_year1,month = month_2+1).values.min()/1000000
            target_irrigation = class_representative_values(target_irrigation, irrigation_demand_classes1,irrigation_demand_representative_values1)
            if storage_list1[month_2] < target_irrigation:
                irrigation_failure = irrigation_failure+1
            if storage_list1[month_2] < target_irrigation and storage_list2[i+1] >= target_irrigation:
                irrigation_failure_sucess = irrigation_failure_sucess + 1

        resiliency_1_irrigation = (irrigation_failure/len(storage_list2))
        resiliency_2_irrigation = (irrigation_failure_sucess/len(storage_list2))

        if resiliency_1_irrigation == 0:
            resiliency_irrigation = 0
        else:
            resiliency_irrigation = resiliency_2_irrigation/resiliency_1_irrigation
        if math.isnan(resiliency_irrigation):
            resiliency_irrigation = 0
        resiliency_irrigation_list.append(resiliency_irrigation)
        
        ################################# Industrial water resiliency #############################   
    resiliency_industrial_list =[]
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['Industrial_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values

        industrial_failure = 0
        industrial_failure_sucess = 0
        for i in range(0,len(storage_list2)-1):
            if storage_list2[i] < Industrial_monthly_Demand:
                industrial_failure = industrial_failure+1

            if storage_list2[i] < Industrial_monthly_Demand and storage_list2[i+1] >= Industrial_monthly_Demand:
                industrial_failure_sucess = industrial_failure_sucess + 1
        resiliency_1_industrial = (industrial_failure/len(storage_list2))
        print (resiliency_1_industrial)
        resiliency_2_industrial = (industrial_failure_sucess/len(storage_list2))
        print(resiliency_2_industrial)
        if resiliency_1_industrial == 0:
            resiliency_industrial = 0
        else:
            resiliency_industrial = resiliency_2_industrial/resiliency_1_industrial
        resiliency_industrial_list.append(resiliency_industrial)
        
        ################################# Hydropower resiliency#############################   
    resiliency_power_full_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand = 1450
        power_failure = 0
        power_failure_sucess = 0
        for i in range(0,len(storage_list2)-1):
            if storage_list2[i] < power_Demand:
                power_failure = power_failure+1
            if storage_list2[i] < power_Demand and storage_list2[i+1] < power_Demand :
                power_failure_sucess = power_failure_sucess + 1
        resiliency_1_power = (power_failure/len(storage_list2))
        resiliency_2_power = (power_failure_sucess/len(storage_list2))
        if resiliency_1_power == 0:
            resiliency_power_full_demand = 0
        else:
            resiliency_power_full_demand = resiliency_2_power/resiliency_1_power

        resiliency_power_full_demand_list.append(resiliency_power_full_demand)

                ################################# Hydropower resiliency (half) #############################   
    resiliency_power_half_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand = 1450/2
        power_failure = 0
        power_failure_sucess = 0
        for i in range(0,len(storage_list2)-1):
            if storage_list2[i] < power_Demand:
                power_failure = power_failure+1
            if storage_list2[i] < power_Demand and storage_list2[i+1] < power_Demand :
                power_failure_sucess = power_failure_sucess + 1
        resiliency_1_power = (power_failure/len(storage_list2))
        resiliency_2_power = (power_failure_sucess/len(storage_list2))
        if resiliency_1_power == 0:
            resiliency_power_half_demand = 0
        else:
            resiliency_power_half_demand = resiliency_2_power/resiliency_1_power
        resiliency_power_half_demand_list.append(resiliency_power_half_demand)

                ################################# Hydropower resiliency (monthly) #############################   
    resiliency_power_monthly_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand_monthly = [250, 250,250, 250,250, 1450,1450,1450,1450, 250,250,250 ]
        power_failure = 0
        power_failure_sucess = 0
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(11):
                if storage_list1[month_2] < power_Demand_monthly[month_2]:
                    power_failure = power_failure +1
                if storage_list1[month_2] < power_Demand_monthly[month_2] and storage_list1[month_2+1] >= power_Demand_monthly[month_2+1]:
                    power_failure_sucess = power_failure_sucess + 1
                start_year1 = start_year1+1
                if start_year1 == 2101:
                    start_year1 = 2100
            month_2 = 11
            if storage_list1[month_2] < power_Demand_monthly[month_2]:
                power_failure = power_failure+1
            if storage_list1[month_2] < power_Demand_monthly[month_2] and storage_list2[month_2+1] >= power_Demand_monthly[0]:
                power_failure_sucess = power_failure_sucess + 1

        resiliency_1_power = (power_failure/len(storage_list2))
        resiliency_2_power = (power_failure_sucess/len(storage_list2))
        if resiliency_1_power == 0:
            resiliency_power_monthly_demand = 0
        else:
            resiliency_power_monthly_demand = resiliency_2_power/resiliency_1_power
        resiliency_power_monthly_demand_list.append(resiliency_power_monthly_demand)

    return [resiliency_flood_list, resiliency_drinking_list, resiliency_irrigation_list, resiliency_industrial_list, resiliency_power_full_demand_list, resiliency_power_half_demand_list, resiliency_power_monthly_demand_list]
      

In [ ]:
def vulneability_flood_1 (storage_list1, target_monthly_storage ):
    j = 0 
    max_values = []
    for k in range(12): 
        #print(k)
        #print(j)
        if k == 0:
            j = 0 
        if k<= j:
            k = j
        if k == len(storage_list1):
            break
            #print(k)
        if k == len(storage_list1)-1:            
            if storage_list1[k] > target_monthly_storage[k]:
                        #print(1)
                j = j+2
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values.append(max_values1)
            else:
                j = j+1
        if k == len(storage_list1)-2: 
            if storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] <= target_monthly_storage[k+1]:
                    #print(1)
                j = j+2
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values.append(max_values1)
                    #print(max_values)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1]:
                    #print(2)
                j = j+3
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
                    #print(max_values)
            else:
                j = j+1
                
        if k == len(storage_list1)-3:
            if storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] <= target_monthly_storage[k+1]:
                    #print(1)
                j = j+2
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values.append(max_values1)
                    #print(max_values)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] <= target_monthly_storage[k+2]:
                j = j+3
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
                    #print(max_values)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] > target_monthly_storage[k+2]:
                    #print(3)
                j = j+4
                    #print(j)
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values3  = storage_list1[k+2] - target_monthly_storage[k+2]
                max_values_max = np.max([max_values1, max_values2, max_values3])
                max_values.append(max_values_max)
            else:
                j = j+1
                
        if k == len(storage_list1)-4: 
            if storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] <= target_monthly_storage[k+1]:
                j = j+2
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values.append(max_values1)
                    #print(max_values)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] <= target_monthly_storage[k+2]:
                    #print(2)
                j = j+3
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
                    #print(max_values)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] <= target_monthly_storage[k+2]:
                j = j+4
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values3  = storage_list1[k+2] - target_monthly_storage[k+2]
                max_values_max = np.max([max_values1, max_values2, max_values3])
                max_values.append(max_values_max)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] > target_monthly_storage[k+2] and storage_list1[k+3] > target_monthly_storage[k+3]:
                j = j+5
                    #print(j)
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values3  = storage_list1[k+2] - target_monthly_storage[k+2]
                max_values4  = storage_list1[k+3] - target_monthly_storage[k+3]
                max_values_max = np.max([max_values1, max_values2, max_values3, max_values4])
                max_values.append(max_values_max)
            else:
                j = j+1
            
        if k < len(storage_list1)-4: 
            if storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] <= target_monthly_storage[k+1]:
                j = j+2
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values.append(max_values1)
                    #print(max_values)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] <= target_monthly_storage[k+2]:
                    #print(2)
                j = j+3
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
                    #print(max_values)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] > target_monthly_storage[k+2]  and storage_list1[k+3] <= target_monthly_storage[k+3]:
                j = j+4
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values3  = storage_list1[k+2] - target_monthly_storage[k+2]
                max_values_max = np.max([max_values1, max_values2, max_values3])
                max_values.append(max_values_max)
            elif storage_list1[k] > target_monthly_storage[k] and storage_list1[k+1] > target_monthly_storage[k+1] and storage_list1[k+2] > target_monthly_storage[k+2] and storage_list1[k+3] > target_monthly_storage[k+3] and storage_list1[k+4] <= target_monthly_storage[k+4]:
                j = j+5
                    #print(j)
                max_values1  = storage_list1[k] - target_monthly_storage[k]
                max_values2  = storage_list1[k+1] - target_monthly_storage[k+1]
                max_values3  = storage_list1[k+2] - target_monthly_storage[k+2]
                max_values4  = storage_list1[k+3] - target_monthly_storage[k+3]
                max_values_max = np.max([max_values1, max_values2, max_values3, max_values4])
                max_values.append(max_values_max)
            else:
                j = j+1
            #print(max_values)
    max_values_mean = np.mean(max_values)
    return max_values_mean

def vulneability_water_demand (storage_list2, drinking_target):
    j = 0 
    max_values = []
    for k in range(len(storage_list2)): 
        #print(k)
        #print(j)
        if k == 0:
            j = 0 
        if k <= j:
            k = j 
        #print(k)
        if k == len(storage_list2):
            break
        if k == len(storage_list2)-1:
            if storage_list2[k] < drinking_target[k]:
                #print(1)
                j = j+2
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values.append(max_values1)
            else:
                j = j+1

        if k == len(storage_list2)-2:   
            if storage_list2[k] < drinking_target[k] and storage_list2[k+1] >= drinking_target[k+1]:
                #print(1)
                j = j+2
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values.append(max_values1)
                        #print(max_values)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(2)
                j = j+3
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
                        #print(max_values)
            else:
                j = j+1

        if k == len(storage_list2)-3:   
            if storage_list2[k] < drinking_target[k] and storage_list2[k+1] >= drinking_target[k+1]:
                #print(1)
                j = j+2
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values.append(max_values1)
                        #print(max_values)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] >= drinking_target[k+2]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(2)
                j = j+3
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] < drinking_target[k+2]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(3)
                j = j+4
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values3  = drinking_target[k+2] - storage_list2[k+2]
                max_values_max = np.max([max_values1, max_values2, max_values3])
                max_values.append(max_values_max)
                        #print(max_values)
            else:
                j = j+1

        if k == len(storage_list2)-4:   
            if storage_list2[k] < drinking_target[k] and storage_list2[k+1] >= drinking_target[k+1]:
                #print(1)
                j = j+2
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values.append(max_values1)
                        #print(max_values)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] >= drinking_target[k+2]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(2)
                j = j+3
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] < drinking_target[k+2] and storage_list2[k+3] >= drinking_target[k+3] : 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(3)
                j = j+4
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values3  = drinking_target[k+2] - storage_list2[k+2]
                max_values_max = np.max([max_values1, max_values2, max_values3])
                max_values.append(max_values_max)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] < drinking_target[k+2] and storage_list2[k+3] < drinking_target[k+3]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(4)
                j = j+5
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values3  = drinking_target[k+2] - storage_list2[k+2]
                max_values4  = drinking_target[k+3] - storage_list2[k+3]
                max_values_max = np.max([max_values1, max_values2, max_values3, max_values4])
                max_values.append(max_values_max)
            else:
                j = j+1

        if k < len(storage_list2)-4:   
            if storage_list2[k] < drinking_target[k] and storage_list2[k+1] >= drinking_target[k+1]:
                #print(1)
                j = j+2
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values.append(max_values1)
                        #print(max_values)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] >= drinking_target[k+2]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(2)
                j = j+3
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values_max = np.max([max_values1, max_values2])
                max_values.append(max_values_max)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] < drinking_target[k+2] and storage_list2[k+3] < drinking_target[k+3]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(3)
                j = j+4
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values3  = drinking_target[k+2] - storage_list2[k+2]
                max_values_max = np.max([max_values1, max_values2, max_values3])
                max_values.append(max_values_max)
            elif storage_list2[k] < drinking_target[k] and storage_list2[k+1] < drinking_target[k+1] and storage_list2[k+2] < drinking_target[k+2] and storage_list2[k+3] < drinking_target[k+3] and storage_list2[k+4] >= drinking_target[k+4]: 
                #and storage_list2[k+2] >= drinking_target[k+2]:
                #print(4)
                j = j+5
                max_values1  = drinking_target[k] - storage_list2[k]
                max_values2  = drinking_target[k+1] - storage_list2[k+1]
                max_values3  = drinking_target[k+2] - storage_list2[k+2]
                max_values4  = drinking_target[k+3] - storage_list2[k+3]
                max_values_max = np.max([max_values1, max_values2, max_values3, max_values4])
                max_values.append(max_values_max)
            else:
                j = j+1
    max_values_mean = np.mean(max_values)
    return max_values_mean


In [ ]:
#define vulneability of the data
def vulneability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_data):
    # select the data between start and end date
    selected_data = full_data.sel(date=slice(start_date, end_date))
################################Flood vulneability #######################################
    vulneability_flood_list = []
    for len_1 in range(len(initial_condition_list)):
        storage_list2 = selected_data['opti_storage'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        max_values_final = []
        for i in range(0,len(storage_list2),12):
            storage_list_1= storage_list2[i:i+12]
            #rint(storage_list1 - target_monthly_storage)
            max_values_1 = vulneability_flood_1 (storage_list_1,target_monthly_storage )
            #print(max_values_1)
           # if len(max_values_1) > 0:
            max_values_final.append(max_values_1)   
        max_values_final = [x for x in max_values_final if math.isnan(x) == False]
        max_values_final1= np.mean(max_values_final)

        vulneability_flood_list.append(max_values_final1)
        
 #################################Drinking water vulneability #############################   
    vulneability_drinking_list =[]
    for len_1 in range(len(initial_condition_list)):
        storage_list2 = selected_data['drinking_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        drinking_target = []
        for i in range(0,len(storage_list2),12):
            target_demand  = (domestic_demand['Drinking_water_demand'].sel(year = start_year1).values)/1000000
            #target_demand = class_representative_values(target_demand, domestic_demand_classes1,domestic_demand_representative_values1)
            start_year1 = start_year1+1
            for k in range(12):
                drinking_target.append(target_demand)
        max_values_mean = vulneability_water_demand (storage_list2, drinking_target)
        if math.isnan(max_values_mean):
            max_values_mean = 0
        vulneability_drinking_list.append(max_values_mean)
        
        ################################  Irrigation water vulneability #############################   
    vulneability_irrigation_list =[]
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['irrigation_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        irrigation_target = []
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(12):
                if start_year1 == 2101:
                    break
                target_irrigation = irrigation_water_demand_data['GIR_crop_volume_cubicmeter'].sel(year = start_year1,month = month_2+1).values.min()/1000000
                target_irrigation = class_representative_values(target_irrigation, irrigation_demand_classes1,irrigation_demand_representative_values1)
                irrigation_target.append(target_irrigation)
            start_year1 = start_year1+1
            print(irrigation_target)
        max_values_mean = vulneability_water_demand (storage_list2, irrigation_target)

        vulneability_irrigation_list.append(max_values_mean)
        
        ################################# Industrial water vulneability #############################   
    vulneability_industrial_list =[]
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['Industrial_water_release'].sel(initial_condition = selected_data['initial_condition'][len_1]).values

        industrial_target = []
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(12):
                if start_year1 == 2101:
                    break
                industrial_target.append(Industrial_monthly_Demand)
        max_values_mean = vulneability_water_demand (storage_list2, industrial_target)
        vulneability_industrial_list.append(max_values_mean)
        
        ################################# Hydropower vulneability #############################   
    vulneability_power_full_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand = 1450
        power_target = []
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(12):
                if start_year1 == 2101:
                    break
                power_target.append(power_Demand)
        max_values_mean = vulneability_water_demand (storage_list2, power_target)
        vulneability_power_full_demand_list.append(max_values_mean)

                ################################# Hydropower vulneability (half) #############################   
    vulneability_power_half_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand = 1450/2
        power_target = []
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(12):
                if start_year1 == 2101:
                    break
                power_target.append(power_Demand)
        max_values_mean = vulneability_water_demand (storage_list2, power_target)
        vulneability_power_half_demand_list.append(max_values_mean)

                ################################# Hydropower vulneability (monthly) #############################   
    vulneability_power_monthly_demand_list = []
    for len_1 in range(len(initial_condition_list)):
        dt = datetime.strptime(start_date, '%Y-%m-%d')
        start_year1 = dt.year
        storage_list2 = selected_data['power_production'].sel(initial_condition = selected_data['initial_condition'][len_1]).values
        power_Demand_monthly = [250, 250,250, 250,250, 1450,1450,1450,1450, 250,250,250 ]
        power_target = []
        for i in range(0,len(storage_list2),12):
            storage_list1 = storage_list2[i:i+12] 
            for month_2 in range(12):
                if start_year1 == 2101:
                    break
                power_target.append(power_Demand_monthly[month_2])
        max_values_mean = vulneability_water_demand (storage_list2, power_target)
        vulneability_power_monthly_demand_list.append(max_values_mean)

    return [vulneability_flood_list, vulneability_drinking_list, vulneability_irrigation_list, vulneability_industrial_list, vulneability_power_full_demand_list, vulneability_power_half_demand_list, vulneability_power_monthly_demand_list]
      

In [ ]:
start_date = '1985-01-1'
end_date = '2101-01-30'
reliability_MME_245_full = reliability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
resiliency_MME_245_full = resiliency_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
vulneability_MME_245_full = vulneability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
# historical data
start_date = '1985-01-1'
end_date = '2015-01-30'
reliability_MME_245_historical = reliability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
resiliency_MME_245_historical = resiliency_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
vulneability_MME_245_historical = vulneability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)

#near term
start_date = '2015-01-1'
end_date = '2045-01-1'
reliability_MME_245_near_term = reliability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
resiliency_MME_245_near_term = resiliency_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
vulneability_MME_245_near_term = vulneability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)

# mid term
start_date = '2045-01-1'
end_date = '2075-01-1'
reliability_MME_245_mid_term = reliability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
resiliency_MME_245_mid_term = resiliency_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
vulneability_MME_245_mid_term = vulneability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)

# far term
start_date = '2075-01-1'
end_date = '2101-01-30'
reliability_MME_245_far_term = reliability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
resiliency_MME_245_far_term = resiliency_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)
vulneability_MME_245_far_term = vulneability_SDP(full_data, start_date, end_date, initial_condition_list, irrigation_water_demand_MME)



In [ ]:
performance_SDP_MME_245 = pd.DataFrame({'initial_condition':initial_condition_list, 'reliability_flood_full':reliability_MME_245_full[0], 'reliability_domestic_full':reliability_MME_245_full[1], 'reliability_irrigation_full':reliability_MME_245_full[2], 'reliability_industrial_full':reliability_MME_245_full[3], 'reliability_power_production_1_full':reliability_MME_245_full[4], 'reliability_power_production_2_full':reliability_MME_245_full[5], 'reliability_power_production_3_full':reliability_MME_245_full[6],
                                       'resiliency_flood_full':resiliency_MME_245_full[0], 'resiliency_domestic_full':resiliency_MME_245_full[1], 'resiliency_irrigation_full':resiliency_MME_245_full[2], 'resiliency_industrial_full':resiliency_MME_245_full[3], 'resiliency_power_production_1_full':resiliency_MME_245_full[4], 'resiliency_power_production_2_full':resiliency_MME_245_full[5], 'resiliency_power_production_3_full':resiliency_MME_245_full[6],
                                       'vulneability_flood_full':vulneability_MME_245_full[0], 'vulneability_domestic_full':vulneability_MME_245_full[1], 'vulneability_irrigation_full':vulneability_MME_245_full[2], 'vulneability_industrial_full':vulneability_MME_245_full[3], 'vulneability_power_production_1_full':vulneability_MME_245_full[4], 'vulneability_power_production_2_full':vulneability_MME_245_full[5], 'vulneability_power_production_3_full':vulneability_MME_245_full[6],
                                       'reliability_flood_near_term':reliability_MME_245_near_term[0], 'reliability_domestic_near_term':reliability_MME_245_near_term[1], 'reliability_irrigation_near_term':reliability_MME_245_near_term[2], 'reliability_industrial_near_term':reliability_MME_245_near_term[3], 'reliability_power_production_1_near_term':reliability_MME_245_near_term[4], 'reliability_power_production_2_near_term':reliability_MME_245_near_term[5], 'reliability_power_production_3_near_term':reliability_MME_245_near_term[6],
                                       'resiliency_flood_near_term':resiliency_MME_245_near_term[0], 'resiliency_domestic_near_term':resiliency_MME_245_near_term[1], 'resiliency_irrigation_near_term':resiliency_MME_245_near_term[2], 'resiliency_industrial_near_term':resiliency_MME_245_near_term[3], 'resiliency_power_production_1_near_term':resiliency_MME_245_near_term[4], 'resiliency_power_production_2_near_term':resiliency_MME_245_near_term[5], 'resiliency_power_production_3_near_term':resiliency_MME_245_near_term[6],
                                       'vulneability_flood_near_term':vulneability_MME_245_near_term[0], 'vulneability_domestic_near_term':vulneability_MME_245_near_term[1], 'vulneability_irrigation_near_term':vulneability_MME_245_near_term[2], 'vulneability_industrial_near_term':vulneability_MME_245_near_term[3], 'vulneability_power_production_1_near_term':vulneability_MME_245_near_term[4], 'vulneability_power_production_2_near_term':vulneability_MME_245_near_term[5], 'vulneability_power_production_3_near_term':vulneability_MME_245_near_term[6],
                                       'reliability_flood_mid_term':reliability_MME_245_mid_term[0], 'reliability_domestic_mid_term':reliability_MME_245_mid_term[1], 'reliability_irrigation_mid_term':reliability_MME_245_mid_term[2], 'reliability_industrial_mid_term':reliability_MME_245_mid_term[3], 'reliability_power_production_1_mid_term':reliability_MME_245_mid_term[4], 'reliability_power_production_2_mid_term':reliability_MME_245_mid_term[5], 'reliability_power_production_3_mid_term':reliability_MME_245_mid_term[6],
                                       'resiliency_flood_mid_term':resiliency_MME_245_mid_term[0], 'resiliency_domestic_mid_term':resiliency_MME_245_mid_term[1], 'resiliency_irrigation_mid_term':resiliency_MME_245_mid_term[2], 'resiliency_industrial_mid_term':resiliency_MME_245_mid_term[3], 'resiliency_power_production_1_mid_term':resiliency_MME_245_mid_term[4], 'resiliency_power_production_2_mid_term':resiliency_MME_245_mid_term[5], 'resiliency_power_production_3_mid_term':resiliency_MME_245_mid_term[6],
                                       'vulneability_flood_mid_term':vulneability_MME_245_mid_term[0], 'vulneability_domestic_mid_term':vulneability_MME_245_mid_term[1], 'vulneability_irrigation_mid_term':vulneability_MME_245_mid_term[2], 'vulneability_industrial_mid_term':vulneability_MME_245_mid_term[3], 'vulneability_power_production_1_mid_term':vulneability_MME_245_mid_term[4], 'vulneability_power_production_2_mid_term':vulneability_MME_245_mid_term[5], 'vulneability_power_production_3_mid_term':vulneability_MME_245_mid_term[6],
                                       'reliability_flood_far_term':reliability_MME_245_far_term[0], 'reliability_domestic_far_term':reliability_MME_245_far_term[1], 'reliability_irrigation_far_term':reliability_MME_245_far_term[2], 'reliability_industrial_far_term':reliability_MME_245_far_term[3], 'reliability_power_production_1_far_term':reliability_MME_245_far_term[4], 'reliability_power_production_2_far_term':reliability_MME_245_far_term[5], 'reliability_power_production_3_far_term':reliability_MME_245_far_term[6],
                                       'resiliency_flood_far_term':resiliency_MME_245_far_term[0], 'resiliency_domestic_far_term':resiliency_MME_245_far_term[1], 'resiliency_irrigation_far_term':resiliency_MME_245_far_term[2], 'resiliency_industrial_far_term':resiliency_MME_245_far_term[3], 'resiliency_power_production_1_far_term':resiliency_MME_245_far_term[4], 'resiliency_power_production_2_far_term':resiliency_MME_245_far_term[5], 'resiliency_power_production_3_far_term':resiliency_MME_245_far_term[6],
                                       'vulneability_flood_far_term':vulneability_MME_245_far_term[0], 'vulneability_domestic_far_term':vulneability_MME_245_far_term[1], 'vulneability_irrigation_far_term':vulneability_MME_245_far_term[2], 'vulneability_industrial_far_term':vulneability_MME_245_far_term[3], 'vulneability_power_production_1_far_term':vulneability_MME_245_far_term[4], 'vulneability_power_production_2_far_term':vulneability_MME_245_far_term[5], 'vulneability_power_production_3_far_term':vulneability_MME_245_far_term[6], 'reliability_flood_historical':reliability_MME_245_historical[0], 'reliability_domestic_historical':reliability_MME_245_historical[1], 'reliability_irrigation_historical':reliability_MME_245_historical[2], 'reliability_industrial_historical':reliability_MME_245_historical[3], 'reliability_power_production_1_historical':reliability_MME_245_historical[4], 'reliability_power_production_2_historical':reliability_MME_245_historical[5], 'reliability_power_production_3_historical':reliability_MME_245_historical[6],
                                       'resiliency_flood_historical':resiliency_MME_245_historical[0], 'resiliency_domestic_historical':resiliency_MME_245_historical[1], 'resiliency_irrigation_historical':resiliency_MME_245_historical[2], 'resiliency_industrial_historical':resiliency_MME_245_historical[3], 'resiliency_power_production_1_historical':resiliency_MME_245_historical[4], 'resiliency_power_production_2_historical':resiliency_MME_245_historical[5], 'resiliency_power_production_3_historical':resiliency_MME_245_historical[6],
                                       'vulneability_flood_historical':vulneability_MME_245_historical[0], 'vulneability_domestic_historical':vulneability_MME_245_historical[1], 'vulneability_irrigation_historical':vulneability_MME_245_historical[2], 'vulneability_industrial_historical':vulneability_MME_245_historical[3], 'vulneability_power_production_1_historical':vulneability_MME_245_historical[4], 'vulneability_power_production_2_historical':vulneability_MME_245_historical[5], 'vulneability_power_production_3_historical':vulneability_MME_245_historical[6]})
dataset_name = path1_SSDP_data +'/Final_figures_SSDP_paper/'+'performance_results_SDP_MME_245_final.nc'
performance_SDP_MME_245.set_index(['initial_condition'], inplace=True)
# Save the dataset to a NetCDF file
performance_SDP_MME_245.to_xarray().to_netcdf(dataset_name)

In [ ]:
# obtain_optimal_pilicies_SDP_MME_SSP245